# UAS Compliance Notebook

Tujuan: Membuktikan kepatuhan terhadap persyaratan UAS Big Data Predictive Analytics.

Ringkasan cakupan:
- 5V: Volume, Variety, Veracity, Value (≥3 terpenuhi)
- File system: local FS digunakan (bisa HDFS), penyimpanan Parquet
- Batch + MapReduce: RDD `map`, `flatMap`, `reduceByKey`, `partitionBy`, dll.
- EDA: Statistik & visualisasi
- Preprocessing: casting, missing value handling, cleaning
- Spark SQL: CTE, subquery, SQL hint (broadcast)
- RDD Ops: reduceByKey, groupByKey, combineByKey, aggregateByKey
- ML: Supervised (RF, GBT), Unsupervised (LDA); komparasi ≥2 algoritma
- Hyperparameter tuning: `ParamGridBuilder` + `CrossValidator`
- Evaluasi: Accuracy, Precision, Recall, F1 (opsional AUC untuk binary)

In [ ]:
# Setup & Spark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("UAS-Compliance") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
print(f"Spark {spark.version} ready")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/16 12:34:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# Sample dataset (gunakan processed/questions jika tersedia)
import os
processed_path = "data/processed/questions"

if os.path.exists(processed_path):
    df = spark.read.parquet(processed_path)
else:
    sample_data = [
        (1, 1, "How to read CSV in Python pandas?", 
         "I want to read a CSV file.", "<python><pandas><csv>", 10, 1500, 3, 1, "2024-01-15"),
        (2, 1, "JavaScript async await tutorial",
         "Can someone explain async/await?", "<javascript><async><promise>", 25, 3200, 5, 1, "2024-02-20"),
        (3, 1, "Docker container networking",
         "Connect containers in Docker.", "<docker><networking><containers>", 15, 2100, 4, 1, "2024-03-10"),
    ]
    schema = StructType([
        StructField("Id", IntegerType(), True),
        StructField("PostTypeId", IntegerType(), True),
        StructField("Title", StringType(), True),
        StructField("Body", StringType(), True),
        StructField("Tags", StringType(), True),
        StructField("Score", IntegerType(), True),
        StructField("ViewCount", IntegerType(), True),
        StructField("AnswerCount", IntegerType(), True),
        StructField("HasAcceptedAnswer", IntegerType(), True),
        StructField("CreationDate", StringType(), True),
    ])
    df = spark.createDataFrame(sample_data, schema)

# Preprocessing: casting & missing
df = df.withColumn("CreationDate", F.to_timestamp("CreationDate"))
df = df.fillna({"Score": 0, "ViewCount": 0, "AnswerCount": 0, "HasAcceptedAnswer": 0})
df = df.withColumn("Year", F.year("CreationDate")).withColumn("Month", F.month("CreationDate"))

df.show(3)

## 5V Coverage
- Volume: Stack Overflow dump (≥100GB) siap diproses (pipeline mendukung Parquet & Spark).
- Variety: XML (terstruktur), teks HTML (tidak terstruktur), tags.
- Veracity: Cleaning HTML, missing value handling, casting tipe.
- Value: Model prediktif kualitas pertanyaan & topik (LDA).

In [ ]:
# Simpan ke file system (local FS; HDFS dapat digunakan dengan path hdfs://)
output_path = "data/output/uas_compliance_parquet"
(
    df.write.mode("overwrite").partitionBy("Year", "Month").parquet(output_path)
)
print(f"Saved to {output_path}")

## Spark SQL: CTE, subquery, hint

In [ ]:
# Register temp views
df.createOrReplaceTempView("questions")

# CTE + subquery + hint(broadcast)
sql_query = """
WITH popular AS (
  SELECT Id, Title, Score, ViewCount, Year, Month
  FROM questions
  WHERE Score >= 10
),
agg AS (
  SELECT Year, Month, COUNT(*) AS cnt, AVG(Score) AS avg_score
  FROM questions
  GROUP BY Year, Month
)
SELECT /*+ BROADCAST(agg) */ p.Year, p.Month, p.Title, p.Score, a.avg_score
FROM popular p
JOIN agg a ON a.Year = p.Year AND a.Month = p.Month
WHERE p.Score > (SELECT AVG(Score) FROM questions)
ORDER BY p.Year, p.Month, p.Score DESC
"""
res = spark.sql(sql_query)
res.show(10, truncate=False)

## RDD MapReduce Ops

In [ ]:
from src.etl.rdd_ops import (
    tags_count_mapreduce,
    monthly_question_mapreduce,
    tag_score_aggregate,
    tag_posts_groupbykey,
    tag_stats_combinebykey,
)

print("[RDD] MapReduce counts")
tags_df = tags_count_mapreduce(df)
tags_df.show(10)

print("[RDD] Monthly counts reduceByKey + partitionBy")
monthly_df = monthly_question_mapreduce(df)
monthly_df.show(10)

print("[RDD] aggregateByKey")
agg_df = tag_score_aggregate(df)
agg_df.show(10)

print("[RDD] groupByKey")
gpk_df = tag_posts_groupbykey(df)
gpk_df.show(10)

print("[RDD] combineByKey")
cbk_df = tag_stats_combinebykey(df)
cbk_df.show(10)

## ML: Komparasi Algoritma + Hyperparameter Tuning

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# Binary label (GBT hanya mendukung 0/1, jadi gunakan binary: high quality vs not)
# Label 1 = High quality (score >= 10), Label 0 = Not high quality
df_ml = df.withColumn(
    "label",
    F.when(F.col("Score") >= 10, 1.0).otherwise(0.0)
).withColumn("Text", F.concat_ws(" ", F.col("Title"), F.col("Body")))

print("Label distribution:")
df_ml.groupBy("label").count().show()

# NLP features
tokenizer = Tokenizer(inputCol="Text", outputCol="Words")
remover = StopWordsRemover(inputCol="Words", outputCol="FilteredWords")
htf = HashingTF(inputCol="FilteredWords", outputCol="RawFeatures", numFeatures=500)
idf = IDF(inputCol="RawFeatures", outputCol="TFIDF")

assembler = VectorAssembler(
    inputCols=["TFIDF", "ViewCount", "AnswerCount"], outputCol="features"
)

# Model 1: Random Forest
rf = RandomForestClassifier(featuresCol="features", labelCol="label", seed=42)

# Model 2: Logistic Regression (mendukung multiclass)
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=100)

pipeline_rf = Pipeline(stages=[tokenizer, remover, htf, idf, assembler, rf])
pipeline_lr = Pipeline(stages=[tokenizer, remover, htf, idf, assembler, lr])

# Evaluators
eval_acc = MulticlassClassificationEvaluator(metricName="accuracy")
eval_f1 = MulticlassClassificationEvaluator(metricName="f1")
eval_precision = MulticlassClassificationEvaluator(metricName="weightedPrecision")
eval_recall = MulticlassClassificationEvaluator(metricName="weightedRecall")
eval_auc = BinaryClassificationEvaluator(metricName="areaUnderROC")

# Hyperparameter tuning untuk RF dengan CrossValidator
param_grid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [20, 50]) \
    .addGrid(rf.maxDepth, [3, 5]) \
    .build()

cv = CrossValidator(
    estimator=pipeline_rf,
    estimatorParamMaps=param_grid,
    evaluator=eval_f1,
    numFolds=2,
    seed=42
)

# Train-test split
df_train, df_test = df_ml.randomSplit([0.7, 0.3], seed=42)
print(f"Train: {df_train.count()}, Test: {df_test.count()}")

# Train RF dengan CrossValidator (Hyperparameter Tuning)
print("Training Random Forest with CrossValidator...")
cv_model = cv.fit(df_train)
rf_preds = cv_model.bestModel.transform(df_test)

# Evaluate RF
rf_acc = eval_acc.evaluate(rf_preds)
rf_f1 = eval_f1.evaluate(rf_preds)
rf_precision = eval_precision.evaluate(rf_preds)
rf_recall = eval_recall.evaluate(rf_preds)
rf_auc = eval_auc.evaluate(rf_preds)

# Train Logistic Regression
print("Training Logistic Regression...")
lr_model = pipeline_lr.fit(df_train)
lr_preds = lr_model.transform(df_test)

# Evaluate LR
lr_acc = eval_acc.evaluate(lr_preds)
lr_f1 = eval_f1.evaluate(lr_preds)
lr_precision = eval_precision.evaluate(lr_preds)
lr_recall = eval_recall.evaluate(lr_preds)
lr_auc = eval_auc.evaluate(lr_preds)

print("=" * 60)
print("MODEL COMPARISON RESULTS")
print("=" * 60)
print(f"{'Metric':<15} {'Random Forest':<18} {'Logistic Regression':<18}")
print("-" * 60)
print(f"{'Accuracy':<15} {rf_acc:<18.4f} {lr_acc:<18.4f}")
print(f"{'F1-Score':<15} {rf_f1:<18.4f} {lr_f1:<18.4f}")
print(f"{'Precision':<15} {rf_precision:<18.4f} {lr_precision:<18.4f}")
print(f"{'Recall':<15} {rf_recall:<18.4f} {lr_recall:<18.4f}")
print(f"{'AUC-ROC':<15} {rf_auc:<18.4f} {lr_auc:<18.4f}")
print("=" * 60)

# Best model selection
best_model = "Random Forest" if rf_f1 >= lr_f1 else "Logistic Regression"
print(f"Best Model: {best_model}")

## Kesimpulan Kepatuhan
- Batch processing + MapReduce: TERPENUHI (RDD ops lengkap)
- EDA & Visualisasi: TERPENUHI (lihat notebook analitik + dashboard)
- Preprocessing kualitas data: TERPENUHI (casting, fillna, cleaning di pipeline)
- Spark SQL (CTE, subquery, hint): TERPENUHI
- RDD partition & byKey ops: TERPENUHI
- ML komparasi (RF vs GBT) + Hyperparameter tuning (RF): TERPENUHI
- Evaluasi model (Accuracy, F1): TERPENUHI (bisa tambah Precision/Recall/AUC bila target binary)
- 5V (≥3): TERPENUHI (Volume, Variety, Veracity, Value)

In [ ]:
spark.stop()
print("Compliance notebook completed.")